## Calculating likelihoods

In [1]:
import matplotlib.pyplot as plt
import corner as corner
import pandas as pd
import numpy as np
import h5py
import torch

from populations.bbh_models import get_models
import populations.bbh_models as read_models
from populations.utils.flow import NFlow
from populations.Flowsclass_dev import FlowModel
from populations import gw_obs

glasflow is using its own internal version of nflows


PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


In [2]:
#reading the pop synth file functions
def get_model_keys(path):
    alpha_val = '10'
    all_models = []
    models = []
    def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])
            
    f = h5py.File(path, 'r')
    f.visititems(find_submodels)
    # get all unique models
    all_models = sorted(list(set(all_models)))
    f.close()

    # use only models with given alpha value
    for model in all_models:
        if 'alpha' in model:
            if 'alpha'+alpha_val in model:
                models.append('/'+model)
        else:
            models.append('/' + model)
    return(np.split(np.array(models), 5))

def get_model_keys_CE(path):
    all_models = []
    models = []
    def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])
            
    f = h5py.File(path, 'r')
    f.visititems(find_submodels)
    # get all unique models
    all_models = sorted(list(set(all_models)))
    return(np.split(np.array(models), 4))

def read_hdf5(path, all_alpha=False):
    if all_alpha:
        popsynth_outputs = {}
        models = np.asarray(get_model_keys_CE(path))
        for i in range(models.shape[0]):
            for j in range(models.shape[1]):
                popsynth_outputs[i,j]=pd.read_hdf(path, key=models[i,j])
    else:
        popsynth_outputs = {}
        models = np.asarray(get_model_keys(path))
        for i in range(models.shape[0]):
            for j in range(models.shape[1]):
                popsynth_outputs[i,j]=pd.read_hdf(path, key=models[i,j])

    return(popsynth_outputs)

In [3]:
params = ['mchirp','q', 'chieff', 'z']

In [4]:
file_path='/Users/stormcolloms/Documents/PhD/Project_work/OneChannel_Flows/models_reduced.hdf5'
gw_path = '/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/gw_events'
observations, obsdata, p_theta, events = gw_obs.generate_observations(params, gw_path, \
                                            10000, 'posteriors', None)

popsynth_outputs = read_hdf5(file_path, True)
flow = FlowModel.from_samples('CE', popsynth_outputs, params)
flow.load_model('/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/flow_models/', 'CE')

In [5]:
llks = flow(obsdata,[3,1])

[[1.30963729e-12 6.04910275e-18 5.42495259e-12 ... 2.44816916e-14
  1.74671916e-15 1.98254617e-14]
 [7.38659648e-17 1.36767524e-11 1.28084004e-31 ... 1.15468554e-25
  1.60183521e-29 6.49078393e-13]
 [2.61927120e-08 2.60109355e-05 3.41982127e-06 ... 2.67131095e-10
  7.34623973e-05 1.05066304e-26]
 ...
 [5.81957602e-06 5.06036564e-08 1.75352255e-07 ... 2.24272114e-07
  3.56762627e-07 3.50336116e-08]
 [1.54004916e-11 1.20401224e-11 3.19936395e-12 ... 2.06451626e-14
  4.39105395e-21 1.55142045e-11]
 [2.89605189e-12 2.57599346e-12 2.89012385e-13 ... 1.42683521e-27
  1.89713000e-16 3.52285827e-13]]


In [7]:
llks

array([4.08980210e+00, 4.56521739e-01, 7.48736093e-03, 1.35101179e-01,
       1.89816358e+02, 4.78260870e-01, 9.07809964e-10, 2.14195652e+02,
       2.00434961e+02, 7.04999714e+02, 1.83695662e+01, 1.67173913e+01,
       1.36711538e+00, 2.57160905e-02, 3.06257681e-01, 1.48503326e-12,
       3.71700909e-01, 2.65891709e+00, 1.36506891e-02, 6.56499166e-11,
       4.85790383e-10, 1.61500025e-08, 3.15252227e-03, 7.66264944e-05,
       1.29291885e+01, 2.39171596e-01, 5.03897917e+01, 1.05928955e+02,
       6.34928825e+01, 3.86956955e+01, 5.99173160e-02, 4.73913053e+00,
       1.38124353e-01, 2.39986112e-09, 2.02672421e-09, 2.09566718e-01,
       1.85447627e-01, 1.77630435e+02, 1.34518307e-05, 2.17391304e+02,
       3.00318189e+02, 2.54348014e+00, 5.66638747e-01, 8.07443036e-04,
       2.70749476e-09, 1.08512266e-09])

In [6]:
llks

array([1.81269817e-02, 1.90000000e-03, 3.28436736e-05, 6.35759471e-04,
       8.77612757e-01, 2.60000000e-03, 4.10987269e-12, 9.84700000e-01,
       9.18502106e-01, 3.15220170e+00, 8.87000144e-02, 7.14000000e-02,
       6.58897416e-03, 1.13040717e-04, 1.40436568e-03, 7.33515191e-15,
       1.30049765e-03, 1.49035317e-02, 6.09099722e-05, 3.01603738e-13,
       2.24819986e-12, 1.06945452e-09, 1.86430848e-03, 2.66516948e-07,
       6.68920318e-02, 1.10019558e-03, 2.36000529e-01, 4.86794876e-01,
       2.83653563e-01, 1.75300199e-01, 2.62889893e-04, 2.39000000e-02,
       6.00281559e-04, 1.12124572e-11, 3.23700801e-11, 9.41189974e-04,
       8.70244088e-04, 8.14000000e-01, 7.05025238e-08, 1.00000000e+00,
       1.40085725e+00, 1.25000062e-02, 3.71521066e-03, 3.84153385e-06,
       1.08873399e-11, 5.08998639e-12])